## Musinsa Customal's Viewpoint - Recommandation System

    작성자 : 조예슬
    
    고객의 구매 목록을 분석하여 상품 추천 시스템 만들기
    
    - MySQL을 이용하여 데이터 전처리 진행
    - 무신사에서 상품 구입이 제일 많은 소비자 선정 후, 가장 많이산 소분류 알아보기 
    - 해당 소분류를 가장 많이 판매한 브랜드 알아보기
    - 위 브랜드들의 브랜드 랭킹 알아보고 상위 3개의 브랜드만 뽑기
    - 상위 3개의 브랜드에서 상위 3개의 제품 추천

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt                                                 # 실제로 그래프를 그려주는 애
import seaborn as sns

import matplotlib.font_manager as fm                                            # 한글폰트 안깨져서 나오게하기위해
font_name = fm.FontProperties(fname="C:\Windows/Fonts/malgun.ttf").get_name()    # 폰트 지정
plt.rc('font', family=font_name)                                                 # matplotlib내에 폰트 설정

mpl.rcParams["axes.unicode_minus"] = False        # 음수 설정, False로 지정하면 음수를 문자로 쓰지않고 숫자로 지정하겠다는 의미, 음수로 사용이 가능해진다.

import MySQLdb

In [4]:
config = {"host":"192.168.0.16", "user":"musinsa", "password":"1111", "database":"PR_TEST", "charset":'utf8'}
conn = MySQLdb.connect(**config)

In [5]:
# 제일 많이 산 소비자(ZENG) SUB_CODE 및 SUB_CODE별 개수 알아보기 

cursor = conn.cursor()  

sql = """SELECT AA.USER_NICK, BB.MAJOR, BB.SUB_CODE, COUNT(DISTINCT AA.PRDT_CODE) AS COUNTS
FROM (SELECT A.USER_NICK, A.PRDT_CODE, B.SUB_CODE
		FROM PRDT_REVIEW A, PRDT_INFO B 
		WHERE A.PRDT_CODE = B.PRDT_CODE) AA,
		CATEGORY BB
WHERE AA.SUB_CODE = BB.SUB_CODE
		AND AA.USER_NICK = "ZENG"
GROUP BY BB.MAJOR, BB.SUB_CODE
ORDER BY COUNTS DESC, BB.SUB_CODE;"""
                      
cursor.execute(sql) 

user1 = cursor.fetchall()

user1_col = ["USER_NICK", "MAJOR", "SUB", "COUNTS"]

USER1 = pd.DataFrame(user1, columns=user1_col)

# # string -> float
# USER1["COUNTS"] = USER1["COUNTS"].astype(float)

USER1 

,USER_NICK,MAJOR,SUB,COUNTS
0,ZENG,상의,001005,23
1,ZENG,상의,001006,23
2,ZENG,모자,007001,10
3,ZENG,상의,001004,9
4,ZENG,상의,001002,8
5,ZENG,상의,001010,8
6,ZENG,상의,001001,5
7,ZENG,가방,004003,5
8,ZENG,아우터,002017,4
9,ZENG,아우터,002003,3


In [6]:
# ZENG의 가장 많이 산 SUB_CODE 뽑기

USER2 = pd.DataFrame(USER1.ix[USER1["COUNTS"].argmax()]).T
USER2

C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  This is separate from the ipykernel package so we can avoid doing imports until


,USER_NICK,MAJOR,SUB,COUNTS
0,ZENG,상의,001005,23


In [7]:
# ZENG SUB_CODE 변수 지정 

sub = USER2["SUB"][0]
sub

'001005'

In [8]:
# 브랜드들의 해당 SUB_CODE(=001005) 판매개수 알기

cursor = conn.cursor()  

sql = """SELECT BRD_NAME_KOR, SUB_CODE, COUNT(PRDT_CODE) AS P_COUNT
FROM PRDT_INFO
GROUP BY BRD_NAME_KOR, SUB_CODE
ORDER BY BRD_NAME_KOR, P_COUNT DESC, SUB_CODE;"""
                      
cursor.execute(sql) 

brand = cursor.fetchall()

brand_col = ["BRD_NAME", "SUB", "P_COUNTS"]

BRAND = pd.DataFrame(brand, columns=brand_col)

# # string -> float
# USER1["COUNTS"] = USER1["COUNTS"].astype(float)

BRAND.head(20)

,BRD_NAME,SUB,P_COUNTS
0,,018002,2
1,(407)HAUS,001010,1
2,090FACTORY,011005,28
3,090FACTORY,011001,18
4,090FACTORY,011003,17
5,090FACTORY,011002,10
6,1'SOFT,001006,5
7,1'SOFT,003008,5
8,1'SOFT,002017,3
9,1'SOFT,002007,2


In [9]:
# 브랜드 주력 상품 코드 갖고 오기 

B = BRAND.ix[BRAND.groupby("BRD_NAME").apply(lambda BRAND:BRAND["P_COUNTS"].argmax())]
B

C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\acorn\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  This is separate from the ipykernel package so we can avoid doing imports until


,BRD_NAME,SUB,P_COUNTS
0,,018002,2
1,(407)HAUS,001010,1
2,090FACTORY,011005,28
6,1'SOFT,001006,5
17,100%,009001,15
...,...,...,...
17317,"sorry, too much love",007005,6
19157,triplePAM,011002,20
20702,willicot,004005,3
20846,wyse,001004,17


In [10]:
# ZENG 001005에 해당하는 brand들 다 가져오기 

S_B = pd.merge(USER2, B, on="SUB", how="left")
S_B

,USER_NICK,MAJOR,SUB,COUNTS,BRD_NAME,P_COUNTS
0,ZENG,상의,001005,23,87MM,21
1,ZENG,상의,001005,23,A-LABEL,3
2,ZENG,상의,001005,23,ABCDE,3
3,ZENG,상의,001005,23,ABNORMALTHING,15
4,ZENG,상의,001005,23,ACIDBLACK,22
...,...,...,...,...,...,...
134,ZENG,상의,001005,23,WYSE,8
135,ZENG,상의,001005,23,XCXMAIN,73
136,ZENG,상의,001005,23,YOU NEED GARMENTS,3
137,ZENG,상의,001005,23,ZEROSTREET,10


In [11]:
# 브랜드 평균 랭킹 : BRAND_RANK 사용

cursor = conn.cursor()  

sql = """SELECT B.BRD_NAME_ENG, ROUND(AVG(A.BRD_RANK), 2) AS AVG_BR
FROM BRAND_RANK A, BRAND_LIST B
WHERE A.RANK_SORT = "POP"
		AND A.RANK_RANGE = "1d"
		AND A.BRD_CODE = B.BRD_CODE
GROUP BY B.BRD_NAME_ENG
ORDER BY AVG_BR ASC;"""
                      
cursor.execute(sql) 

brand_R = cursor.fetchall()

brand_R_col = ["BRD_NAME", "AVG_BR"]

BRAND_R = pd.DataFrame(brand_R, columns=brand_R_col)

# # string -> float
# USER1["COUNTS"] = USER1["COUNTS"].astype(float)

BRAND_R

,BRD_NAME,AVG_BR
0,MUSINSA STANDARD,1.00
1,COVERNAT,2.29
2,ADIDAS,3.29
3,NIKE,5.43
4,THISISNEVERTHAT,5.57
...,...,...
117,MIGNONNEUF,97.00
118,URBANSTOFF,97.25
119,AMBLER,97.75
120,23.65,99.00


In [12]:
# # ZENG 001005에 해당하는 brand들의 랭킹 

S_B_R = pd.merge(S_B, BRAND_R, on="BRD_NAME", how="left")
S_B_R2 = S_B_R.dropna(axis =0)
S_B_R3 = S_B_R2.sort_values(by=["AVG_BR"])
S_B_R3

,USER_NICK,MAJOR,SUB,COUNTS,BRD_NAME,P_COUNTS,AVG_BR
27,ZENG,상의,001005,23,COVERNAT,100,2.29
119,ZENG,상의,001005,23,THISISNEVERTHAT,70,5.57
23,ZENG,상의,001005,23,COMPAGNO,86,23.14
130,ZENG,상의,001005,23,VIVASTUDIO,20,25.00
76,ZENG,상의,001005,23,MARK GONZALES,41,27.86
122,ZENG,상의,001005,23,TRAVEL,53,39.00
58,ZENG,상의,001005,23,JEEP,49,47.43
41,ZENG,상의,001005,23,FRIZMWORKS,25,48.29
102,ZENG,상의,001005,23,ROMANTIC CROWN,59,50.71
10,ZENG,상의,001005,23,ANDERSSON BELL,22,61.14


In [14]:
# ZENG SUB_CODE 변수 지정 

sub = USER2["SUB"][0]
sub

'001005'

In [27]:
# DataFrame 이용해서 브랜드마다 상품 3개씩 뽑아내기

# 빈 DataFrame 만들기
df_brand = pd.DataFrame()
df_brand

# for문
for i in range(0,3) :
    B_name = S_B_R3.iloc[i,4]
    
    sql = """SELECT AA.BRD_NAME_KOR, AA.SUB_CODE, AA.PRDT_NAME, BB.PUR_CNT
    FROM(SELECT B.BRD_NAME_KOR, B.SUB_CODE, A.PRDT_NAME, A.PRDT_CODE
          FROM RANK_ALL A, PRDT_INFO B
          WHERE A.PRDT_CODE = B.PRDT_CODE
            AND B.BRD_NAME_KOR = '{}'
            AND B.SUB_CODE = '{}'
            LIMIT 3) AA,
            PRDT_PURCHASE BB
    WHERE AA.PRDT_CODE = BB.PRDT_CODE
      AND BB.PUR_CNT <= 100000
    ORDER BY BB.PUR_CNT DESC;""".format(B_name, sub)
                      
    cursor.execute(sql) 
    
    PRDT = cursor.fetchall()
    
    PRDT_col = ["BRD_NAME", "SUB", "PRDT_NAME", "PUR_CNT"]

    PRDT = pd.DataFrame(PRDT, columns=PRDT_col)    
    
    df_brand = pd.concat([df_brand, PRDT])
    
df_brand

,BRD_NAME,SUB,PRDT_NAME,PUR_CNT
0,COVERNAT,001005,1/28 배송 COVERNAT x M/G LAUNDRY LOGO CREWNECK I...,0
1,COVERNAT,001005,COOPER LOGO CREWNECK NAVY,0
2,COVERNAT,001005,LAYOUT LOGO CREWNECK BLACK,0
0,THISISNEVERTHAT,001005,1/6 배송 T-Logo Crewneck Black,0
1,THISISNEVERTHAT,001005,1/6 배송 SP-Logo Crewneck Black,0
2,THISISNEVERTHAT,001005,1/6 배송 T-Logo Crewneck Charcoal,0
0,COMPAGNO,001005,[방탄소년단 뷔][기모] MTM 피에로 자수 헤비오버핏 브라운,3952
1,COMPAGNO,001005,[정은지][기모] James Madison 크레용 맨투맨 브라운,226
2,COMPAGNO,001005,[기모] MTM 헤리티지 월계수 자수 그레이,218


In [68]:
## 번외(list와 dataframe 차이점 알아보기)
# list 이용해서 브랜드마다 상품 3개씩 뽑아내기

cursor = conn.cursor()  

sql = """SELECT AA.BRD_NAME_KOR, AA.SUB_CODE, AA.PRDT_NAME, BB.PUR_CNT
    FROM(SELECT B.BRD_NAME_KOR, B.SUB_CODE, A.PRDT_NAME, A.PRDT_CODE
          FROM RANK_ALL A, PRDT_INFO B
          WHERE A.PRDT_CODE = B.PRDT_CODE
            AND B.BRD_NAME_KOR IN ('COVERNAT', 'THISISNEVERTHAT')
            AND B.SUB_CODE = '{}') AA,
            PRDT_PURCHASE BB
    WHERE AA.PRDT_CODE = BB.PRDT_CODE
      AND BB.PUR_CNT <= 100000
    ORDER BY BB.PUR_CNT DESC
    LIMIT 3;""".format(sub)
                      
cursor.execute(sql) 

PRDT = cursor.fetchall()

PRDT_col = ["BRD_NAME", "SUB", "PRDT_NAME", "PUR_CNT"]

PRDT = pd.DataFrame(PRDT, columns=PRDT_col)

# # string -> float
# USER1["COUNTS"] = USER1["COUNTS"].astype(float)

PRDT

,BRD_NAME,SUB,PRDT_NAME,PUR_CNT
0,COVERNAT,001005,[아이즈원]C LOGO CREWNECK GREEN,0
1,COVERNAT,001005,COTTON USA WAPPEN ARCH LOGO CREWNECK BLACK,0
2,COVERNAT,001005,EMBROIDERY JERSEY LOGO CREWNECK BLACK,0


In [13]:
# list에 브랜드 상위 3개  이름 넣기

list_brand = list()

for i in range(0,3) :
    a = S_B_R3.iloc[i,4]
    
    list_brand.append(a)
    
list_brand

['COVERNAT', 'THISISNEVERTHAT', 'COMPAGNO']

In [92]:
# list 이용해서 브랜드마다 상품 3개씩 뽑아내기

## 상위 3개 브랜드 나열 list로 
list_brand = list()

for i in range(0,3) :
    a = S_B_R3.iloc[i,4]
    
    list_brand.append(a)
    
list_brand

## sql에 list 대입 시키기 

for i in list_brand :
    print(i)

    sql = """SELECT AA.BRD_NAME_KOR, AA.SUB_CODE, AA.PRDT_NAME, BB.PUR_CNT
        FROM(SELECT B.BRD_NAME_KOR, B.SUB_CODE, A.PRDT_NAME, A.PRDT_CODE
              FROM RANK_ALL A, PRDT_INFO B
              WHERE A.PRDT_CODE = B.PRDT_CODE
                AND B.BRD_NAME_KOR = '{}'
                AND B.SUB_CODE = '{}'
              LIMIT 3) AA,
                PRDT_PURCHASE BB
        WHERE AA.PRDT_CODE = BB.PRDT_CODE
          AND BB.PUR_CNT <= 100000
        ORDER BY BB.PUR_CNT DESC;""".format(i, sub)
                      
    cursor.execute(sql) 

    PRDT = cursor.fetchall()

    PRDT_col = ["BRD_NAME", "SUB", "PRDT_NAME", "PUR_CNT"]

    PRDT = pd.DataFrame(PRDT, columns=PRDT_col)
    
    

# # string -> float
# USER1["COUNTS"] = USER1["COUNTS"].astype(float)

PRDT

COVERNAT
THISISNEVERTHAT
COMPAGNO


,BRD_NAME,SUB,PRDT_NAME,PUR_CNT
0,COMPAGNO,001005,[방탄소년단 뷔][기모] MTM 피에로 자수 헤비오버핏 브라운,3952
1,COMPAGNO,001005,[정은지][기모] James Madison 크레용 맨투맨 브라운,226
2,COMPAGNO,001005,[기모] MTM 헤리티지 월계수 자수 그레이,218


In [ ]:
# for문 안에 sql을 쓰기
# sql바로 밑에 cursor.excute
# cursor.fetchall 까지 for문 안에 있어야함
# 3브랜드를 담아놓을 변수가 필요함 

# for문 돌리면 각브랜드당 상품3개 
# sql문에 limit(3)하던가 
# 나중에 head(3)

# 나중에 9개 한번에 뽑아 낼때는 limit(3)
# 나중에 head(9)하던가 

### 최종결론

    무신사에서 상품 구입이 제일 많은 소비자는 : ZENG
    ZENG이 가장 많이 산 소분류 : 001005
    001005가 가장 많이 팔리는 상위 랭킹 3 브랜드는 : COVERNAT, THISISNEVERTHAT, COMPAGNO
    
    - DataFrame과 List의 차이점
    DataFrame으로 상품 9개를 뽑으면 각 브랜드 마다 3개씩 상품 추천
    List로 상품 9개를 뽑으면 3개의 브랜드 안에서 가장 많이 팔린 상품 9개 추천
    브랜드 개수 구별 없이 상품이 뽑힘